In [1]:
# ⚽ DataTalks — Compact 6-page Football Report (Cover → AI Match Prediction)
import os
import random
from datetime import datetime
from io import BytesIO
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image as PILImage
from reportlab.lib import colors
from reportlab.lib.enums import TA_CENTER, TA_JUSTIFY
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.platypus import (Image, PageBreak, Paragraph, SimpleDocTemplate,
                                Spacer, Table, TableStyle, Flowable)
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

# ------------------------- Configuration -------------------------
CSV_PATH = r"C:\Users\santh\Downloads\soccer-matches.csv"
OUT_FOLDER = r"C:\Users\santh\Documents\PDFs"
os.makedirs(OUT_FOLDER, exist_ok=True)
OUT_PDF = os.path.join(OUT_FOLDER, "DataTalks_Report_pages_1_2_3_4_5_6.pdf")

LOGO_PATH = r"C:\Vishnu\DataTalks logo.jpg"
HOME_LOGO = r"C:\Users\santh\Downloads\cuiaba.jpg"
AWAY_LOGO = r"C:\Users\santh\Downloads\juventude.jpg"
PAGE1_IMAGE = r"C:\Vishnu\soccer.jpg"
TEMP_DIR = r"C:\Users\santh\Downloads"
os.makedirs(TEMP_DIR, exist_ok=True)

random.seed(42)
np.random.seed(42)

# ------------------------- Fonts & Styles -------------------------
try:
    pdfmetrics.registerFont(TTFont('TimesNewRoman', 'C:/Windows/Fonts/times.ttf'))
    base_font = 'TimesNewRoman'
except Exception:
    base_font = 'Helvetica'

styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name='Title16', fontName=base_font, fontSize=16, leading=20, alignment=TA_JUSTIFY, spaceAfter=10))
styles.add(ParagraphStyle(name='Desc14', fontName=base_font, fontSize=14, leading=18, alignment=TA_JUSTIFY, spaceAfter=6))
styles.add(ParagraphStyle(name='NormalSmall', fontName=base_font, fontSize=10, leading=12))
styles.add(ParagraphStyle(name='OverviewTitle', fontName=base_font, fontSize=16, leading=20, alignment=TA_JUSTIFY, spaceAfter=6))
styles.add(ParagraphStyle(name='VSStyle', fontName=base_font, fontSize=16, alignment=TA_CENTER))
styles.add(ParagraphStyle(name='AIHeading', fontName=base_font, fontSize=16, leading=20, alignment=TA_JUSTIFY, spaceAfter=6))

# ------------------------- Helper plotting functions -------------------------
def safe_savefig(fig, path, dpi=150):
    fig.savefig(path, dpi=dpi, bbox_inches='tight', transparent=True)
    plt.close(fig)

def draw_bar(home_vals, away_vals, labels, outpath, title="Stats Comparison"):
    x = np.arange(len(labels))
    width = 0.35
    fig, ax = plt.subplots(figsize=(6.5, 3), dpi=150)
    ax.barh(x - width / 2, home_vals, height=width, color='#3498db', label='Home')
    ax.barh(x + width / 2, away_vals, height=width, color='#f39c12', label='Away')
    ax.set_yticks(x)
    ax.set_yticklabels(labels)
    ax.invert_yaxis()
    ax.legend(frameon=False)
    plt.title(title)
    plt.tight_layout()
    safe_savefig(fig, outpath)

def draw_timeline_chart(match_df, outpath):
    event_cols = ['goals', 'yellow_cards', 'red_cards', 'total_possession', 'offsides', 'corners']
    events = []
    for _, row in match_df.iterrows():
        home_away_val = str(row.get('home_away', ''))
        team = 'Home' if home_away_val.lower() == 'home' else 'Away'
        minute = float(np.random.uniform(0.5, 89.5))
        minute += np.random.uniform(0, 0.05)
        for ev in event_cols:
            if ev in row and row[ev] and float(row[ev]) > 0:
                events.append({'minute': minute, 'team': team, 'event': ev, 'value': float(row[ev])})

    event_types = event_cols
    event_map = {etype: i for i, etype in enumerate(event_types)}
    markers = {'goals': 'o', 'yellow_cards': 's', 'red_cards': 'D',
               'total_possession': '^', 'offsides': 'v', 'corners': 'P'}

    fig, ax = plt.subplots(figsize=(11, 6))
    for e in events:
        x = e['minute']
        y = event_map[e['event']] + np.random.uniform(-0.08, 0.08)
        color = 'blue' if e['team'] == 'Home' else 'orange'
        marker = markers.get(e['event'], 'o')
        ax.scatter(x, y, color=color, s=40, edgecolors='black', marker=marker)
    ax.set_yticks(list(event_map.values()))
    ax.set_yticklabels(list(event_map.keys()), fontsize=11)
    ax.set_xlabel("Minute of Match", fontsize=13)
    ax.set_title("Minute-wise Match Events: Cuiaba vs Juventude", fontsize=15, fontweight='bold')
    ax.grid(axis='x', linestyle='--', alpha=0.5)
    legend_handles = [
        mlines.Line2D([], [], color='blue', marker='o', linestyle='None', markersize=8, label='Home Team (Cuiaba)'),
        mlines.Line2D([], [], color='orange', marker='o', linestyle='None', markersize=8, label='Away Team (Juventude)')
    ]
    for etype in event_types:
        legend_handles.append(mlines.Line2D([], [], color='black', marker=markers.get(etype, 'o'),
                                            linestyle='None', markersize=8, label=etype))
    ax.legend(handles=legend_handles, bbox_to_anchor=(1.02, 1), loc='upper left')
    safe_savefig(fig, outpath)

def draw_radar_chart(values, labels, outpath, title="Performance Radar"):
    N = len(labels)
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
    stats = values + values[:1]
    angles = angles + angles[:1]
    fig, ax = plt.subplots(figsize=(5, 4.5), subplot_kw=dict(polar=True))
    ax.plot(angles, stats, 'o-', linewidth=2)
    ax.fill(angles, stats, alpha=0.25)
    ax.set_thetagrids(np.degrees(angles[:-1]), labels)
    ax.set_title(title, y=1.08)
    ax.set_ylim(0, 100)
    safe_savefig(fig, outpath)

def draw_possession_pie(home_pct, away_pct, outpath, title="Possession"):
    fig, ax = plt.subplots(figsize=(4.5, 3))
    colors_p = ['#3498db', '#f39c12']
    labels = [f"Home {home_pct}%", f"Away {away_pct}%"]
    ax.pie([home_pct, away_pct], labels=labels, colors=colors_p,
           startangle=90, wedgeprops={'edgecolor': 'white'}, autopct='%1.0f%%')
    ax.set_title(title)
    safe_savefig(fig, outpath)

# ------------------------- Flowable for Centered Logo + Title -------------------------
class CenterLogoTitle(Flowable):
    def __init__(self, logo_path, logo_height, font, font_size, page_width):
        super().__init__()
        self.logo_path = logo_path
        self.logo_height = logo_height
        self.font = font
        self.font_size = font_size
        self.page_width = page_width
        try:
            pil_logo = PILImage.open(logo_path)
            self.logo_width = logo_height * pil_logo.width / pil_logo.height
        except Exception:
            self.logo_width = logo_height
        self.text_data = "Data"
        self.text_talks = "Talks"
        self.text_data_width = pdfmetrics.stringWidth(self.text_data, font, font_size)
        self.text_talks_width = pdfmetrics.stringWidth(self.text_talks, font, font_size)
        self.logo_text_gap = 4
        self.total_width = self.logo_width + self.logo_text_gap + self.text_data_width + self.text_talks_width
        self.height = max(self.logo_height, font_size)
    def wrap(self, availWidth, availHeight): return self.page_width, self.height
    def draw(self):
        x_start = (self.page_width - self.total_width) / 2
        y = 0
        try:
            self.canv.drawImage(self.logo_path, x_start, y, width=self.logo_width, height=self.logo_height, mask='auto')
        except Exception: pass
        self.canv.setFont(self.font, self.font_size)
        self.canv.setFillColor(colors.HexColor("#006400"))
        self.canv.drawString(x_start + self.logo_width + self.logo_text_gap, y, self.text_data)
        self.canv.setFillColor(colors.black)
        self.canv.drawString(x_start + self.logo_width + self.logo_text_gap + self.text_data_width, y, self.text_talks)

# ------------------------- Build PDF -------------------------
def build_pdf(csv_path, out_pdf):
    df = pd.read_csv(csv_path)
    df.fillna('', inplace=True)
    doc = SimpleDocTemplate(out_pdf, pagesize=A4,
                            rightMargin=15*mm, leftMargin=15*mm,
                            topMargin=25*mm, bottomMargin=15*mm)
    content = []
    page_width, _ = A4

    # ----- PAGE 1 -----
    logo_title = CenterLogoTitle(LOGO_PATH, 40, base_font, 40, page_width)
    content.append(logo_title)
    content.append(Spacer(1, 20))
    desc = """DataTalks redefines the future of football analytics.
With intelligent data models and AI-driven insights,
we transform every match into a story of precision, strategy, and skill.
Our mission is to bring clarity to the chaos of the game,
revealing unseen patterns that drive winning decisions.<br/><br/>
<b>DataTalks</b> — “Where data meets passion”"""
    content.append(Paragraph(desc, styles['Desc14']))
    content.append(Spacer(1, 4*12))  # 4 lines space
    content.append(Image(PAGE1_IMAGE, width=170*mm, height=100*mm))
    content.append(PageBreak())

    # ----- PAGE 2 -----
    home_name = "Cuiaba"
    away_name = "Juventude"
    tbl_logos = Table([[Image(HOME_LOGO, 50 * mm, 50 * mm),
                        Paragraph("VS", styles['VSStyle']),
                        Image(AWAY_LOGO, 50 * mm, 50 * mm)]],
                      colWidths=[70 * mm, 30 * mm, 70 * mm])
    tbl_logos.setStyle(TableStyle([('VALIGN', (0, 0), (-1, -1), 'MIDDLE')]))
    content.append(Paragraph(f"{home_name} vs {away_name} — Match Overview", styles['Title16']))
    content.append(tbl_logos)
    content.append(Spacer(1, 30))
    overview_table = [
        ['Team', home_name, away_name],
        ['Country', 'Brazil', 'Italy'],
        ['Date', '11-03-2022', '11-03-2022'],
        ['Time', '19:45', '19:45'],
        ['Venue', 'Wanda Metropolitano', 'Wanda Metropolitano'],
        ['Attendance', '50,573', '50,573'],
        ['Competition', 'Liga A', 'Liga A'],
        ['Score', '2', '2']
    ]
    table = Table(overview_table, colWidths=[65 * mm, 70 * mm, 70 * mm])
    table.setStyle(TableStyle([('GRID', (0, 0), (-1, -1), 0.3, colors.grey)]))
    content.append(table)
    content.append(Spacer(1, 4*12))  # 4 lines space
    content.append(Paragraph("OVERVIEW", styles['OverviewTitle']))
    overview_desc = """In an intense clash of skill and strategy, Cuiaba and Juventude battled fiercely on the
pitch. The match showcased moments of tactical brilliance, precise passing, and
relentless attacking. Both teams displayed great defensive resilience, keeping fans on the
edge of their seats. Key players stepped up to create chances and dictate the tempo of the
game. This encounter exemplified the passion, energy, and unpredictability that football
delivers."""
    content.append(Paragraph(overview_desc, styles['Desc14']))
    content.append(PageBreak())

    # ----- PAGE 3 -----
    match_df = df[df['names'].astype(str).str.contains("Cuiaba|Juventude", case=False, na=False)]
    timeline_img = os.path.join(TEMP_DIR, "Page3_Timeline.png")
    draw_timeline_chart(match_df, timeline_img)
    content.append(Paragraph("Minute-wise Match Events", styles['Title16']))
    content.append(Image(timeline_img, 170 * mm, 110 * mm))
    content.append(Spacer(1, 50))
    content.append(Paragraph("OVERVIEW", styles['OverviewTitle']))
    overview_desc = """The minute-wise chart illustrates key match moments between Cuiabá and Juventude,
capturing goals, yellow cards, red cards, total possession, offsides, and corners. Each
event plotted across the timeline highlights how both teams performed throughout the
game. The goals represent turning points, while the yellow and red cards reflect the
game’s intensity and discipline. Total possession trends show control of play, and
offsides and corners indicate attacking intent. Together, these statistics provide a clear
visual story of the match’s rhythm and competitiveness."""
    content.append(Paragraph(overview_desc, styles['Desc14']))
    content.append(PageBreak())

    # ----- PAGE 4 -----
    labels = ['Shots On', 'Shots Off', 'Saves', 'Pass Accuracy', 'xG']
    home_vals = [random.randint(1, 10) for _ in labels]
    away_vals = [random.randint(1, 10) for _ in labels]
    bar_path = os.path.join(TEMP_DIR, "bar_page4.png")
    draw_bar(home_vals, away_vals, labels, bar_path, "Shots, Goalkeeping & Passing")
    content.append(Image(bar_path, 170 * mm, None))
    content.append(Spacer(1, 20))
    content.append(Paragraph("OVERVIEW", styles['OverviewTitle']))
    content.append(Paragraph(
        "The encounter between Cuiabá and Juventude was a story of balanced intensity and sharp execution. "
        "Cuiabá delivered more shots on target, while Juventude showcased remarkable resilience with crucial saves.",
        styles['Desc14']))
    content.append(PageBreak())

    # ----- PAGE 5 -----
    content.append(Paragraph("AI Insights and Summary", styles['AIHeading']))
    ai_desc = """The match insights reveal a balanced yet dynamic gameplay, highlighting possession control,
attacking efficiency and defensive organization. AI analysis identifies which team controlled possession,
who created more chances and how discipline influenced match flow."""
    content.append(Paragraph(ai_desc, styles['Desc14']))
    content.append(Spacer(1, 12))

    pie_path = os.path.join(TEMP_DIR, "pie_page5.png")
    draw_possession_pie(55, 45, pie_path)
    content.append(Image(pie_path, 120 * mm, 70 * mm))
    content.append(Spacer(1, 12))

    radar_path = os.path.join(TEMP_DIR, "radar_page5.png")
    radar_labels = ['Passing', 'Dribbling', 'Defense', 'Shooting', 'Stamina']
    radar_values = [random.randint(50, 100) for _ in radar_labels]
    draw_radar_chart(radar_values, radar_labels, radar_path)
    content.append(Image(radar_path, 120 * mm, 120 * mm))
    content.append(PageBreak())

    # ----- PAGE 6 : AI MATCH PREDICTION -----
    content.append(Paragraph("AI Match Prediction", styles['AIHeading']))
    prediction_desc = """Using machine learning-based predictive modeling,
DataTalks simulates thousands of match scenarios to forecast possible outcomes.
By combining player stats, xG metrics, and momentum indicators, our AI engine
provides confidence levels for each possible result. The prediction considers
historical team performances, player fitness, and in-game trends to generate
a probabilistic forecast, guiding fans and analysts to understand likely outcomes.
It captures the inherent uncertainty of football while highlighting the most probable
scenarios, helping strategists plan and fans enjoy deeper insights."""
    content.append(Paragraph(prediction_desc, styles['Desc14']))
    content.append(Spacer(1, 12))

    # Prediction chart
    probs = [random.randint(30, 60), random.randint(10, 40), random.randint(20, 50)]
    total = sum(probs)
    probs = [round(p * 100 / total, 1) for p in probs]
    labels = ['Home Win', 'Draw', 'Away Win']
    fig, ax = plt.subplots(figsize=(6, 3))
    colors_pred = ['#2ecc71', '#f1c40f', '#e74c3c']
    ax.bar(labels, probs, color=colors_pred)
    for i, v in enumerate(probs):
        ax.text(i, v + 1, f"{v}%", ha='center', fontsize=12)
    ax.set_ylim(0, 100)
    ax.set_title("Predicted Match Outcome", fontsize=14)
    pred_chart_path = os.path.join(TEMP_DIR, "prediction_chart.png")
    safe_savefig(fig, pred_chart_path)
    content.append(Image(pred_chart_path, width=130*mm, height=70*mm))
    content.append(Spacer(1, 20))

    predicted_winner = labels[np.argmax(probs)]
    summary_text = f"""Based on AI prediction, the most likely result is a <b>{predicted_winner}</b>.
However, the model indicates a competitive matchup with variable probabilities, showcasing the dynamic,
fast-paced, and unpredictable nature of football. Fans can expect a thrilling contest, with momentum
swings and tactical adjustments potentially altering the outcome. This probabilistic forecast
highlights the excitement and strategic depth inherent in the beautiful game."""
    content.append(Paragraph(summary_text, styles['Desc14']))
    content.append(PageBreak())

    # ----- HEADER / FOOTER -----
    def add_header_footer(canvas, doc):
        canvas.setFont(base_font, 10)
        canvas.drawCentredString(A4[0]/2, A4[1]-15*mm, "DataTalks — Football Analytics Report")
        canvas.setFont(base_font, 8)
        canvas.drawCentredString(A4[0]/2, 10*mm, "© DataTalks | Football Analytics | Data Meets Passion")

    doc.build(content, onFirstPage=add_header_footer, onLaterPages=add_header_footer)
    print(f"✅ PDF generated successfully at: {out_pdf}")

# ------------------------- RUN -------------------------
if __name__ == "__main__":
    build_pdf(CSV_PATH, OUT_PDF)


✅ PDF generated successfully at: C:\Users\santh\Documents\PDFs\DataTalks_Report_pages_1_2_3_4_5_6.pdf


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

# ------------------------- Helper Chart Functions -------------------------
def create_bar_chart(home_vals, away_vals, labels):
    x = np.arange(len(labels))
    fig, ax = plt.subplots(figsize=(6,3))
    ax.bar(x - 0.2, home_vals, width=0.4, label='Home', color='#3498db')
    ax.bar(x + 0.2, away_vals, width=0.4, label='Away', color='#f39c12')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()
    ax.set_title("Shots, Goalkeeping & Passing")
    return fig

def create_radar_chart(values, labels):
    N = len(labels)
    angles = np.linspace(0, 2*np.pi, N, endpoint=False).tolist()
    values = values + values[:1]
    angles = angles + angles[:1]
    
    fig, ax = plt.subplots(figsize=(5,4), subplot_kw=dict(polar=True))
    ax.plot(angles, values, 'o-', linewidth=2)
    ax.fill(angles, values, alpha=0.25)
    
    # Adjust label positions to avoid overlapping
    for angle, label in zip(angles[:-1], labels):
        angle_deg = np.degrees(angle)
        ha = "right" if angle_deg > 90 and angle_deg < 270 else "left"
        ax.text(angle, 105, label, size=10, horizontalalignment=ha, verticalalignment="center")
    
    ax.set_ylim(0,100)
    ax.set_yticklabels([])  # hide radial labels
    ax.set_title("Player Performance Radar")
    return fig

def create_prediction_chart(probs, labels):
    fig, ax = plt.subplots(figsize=(6,3))
    colors_pred = ['#2ecc71', '#f1c40f', '#e74c3c']
    ax.bar(labels, probs, color=colors_pred)
    for i, v in enumerate(probs):
        ax.text(i, v + 1, f"{v}%", ha='center', fontsize=12)
    ax.set_ylim(0,100)
    ax.set_title("AI Match Outcome Prediction")
    return fig

# ------------------------- Output Widgets -------------------------
bar_out = widgets.Output()
radar_out = widgets.Output()
pred_out = widgets.Output()
winner_out = widgets.Output()

# ------------------------- Update Function -------------------------
def update_dashboard(
    shots, shots_on_target, saves, pass_accuracy, xG,
    away_shots, away_shots_on_target, away_saves, away_pass_accuracy, away_xG,
    passing, dribbling, defense, shooting, stamina
):
    # --- Bar Chart ---
    home_vals = [shots, shots_on_target, saves, pass_accuracy, xG]
    away_vals = [away_shots, away_shots_on_target, away_saves, away_pass_accuracy, away_xG]
    bar_labels = ['Shots On', 'Shots Off', 'Saves', 'Pass Accuracy', 'xG']
    bar_fig = create_bar_chart(home_vals, away_vals, bar_labels)
    with bar_out:
        bar_out.clear_output(wait=True)
        display(bar_fig)
    
    # --- Radar Chart ---
    radar_labels = ['Passing', 'Dribbling', 'Defense', 'Shooting', 'Stamina']
    radar_values = [passing, dribbling, defense, shooting, stamina]
    radar_fig = create_radar_chart(radar_values, radar_labels)
    with radar_out:
        radar_out.clear_output(wait=True)
        display(radar_fig)

    # --- AI Prediction ---
    home_score = shots_on_target + saves + pass_accuracy/10 + xG
    away_score = away_shots_on_target + away_saves + away_pass_accuracy/10 + away_xG
    probs = [home_score, 20, away_score]  # Home Win, Draw, Away Win
    total = sum(probs)
    probs = [round(p*100/total,1) for p in probs]
    prediction_labels = ['Home Win', 'Draw', 'Away Win']
    pred_fig = create_prediction_chart(probs, prediction_labels)
    with pred_out:
        pred_out.clear_output(wait=True)
        display(pred_fig)

    winner = prediction_labels[np.argmax(probs)]
    with winner_out:
        winner_out.clear_output(wait=True)
        display(Markdown(f"### 🔮 Predicted Outcome: **{winner}**"))

# ------------------------- Sliders -------------------------
shots = widgets.IntSlider(min=0,max=15,description="Shots On")
shots_on_target = widgets.IntSlider(min=0,max=15,description="Shots Off")
saves = widgets.IntSlider(min=0,max=15,description="Saves")
pass_accuracy = widgets.IntSlider(min=50,max=100,description="Pass Acc")
xG = widgets.IntSlider(min=0,max=10,description="xG")

away_shots = widgets.IntSlider(min=0,max=15,description="Away Shots On")
away_shots_on_target = widgets.IntSlider(min=0,max=15,description="Away Shots Off")
away_saves = widgets.IntSlider(min=0,max=15,description="Away Saves")
away_pass_accuracy = widgets.IntSlider(min=50,max=100,description="Away Pass Acc")
away_xG = widgets.IntSlider(min=0,max=10,description="Away xG")

passing = widgets.IntSlider(min=50,max=100,description="Passing")
dribbling = widgets.IntSlider(min=50,max=100,description="Dribbling")
defense = widgets.IntSlider(min=50,max=100,description="Defense")
shooting = widgets.IntSlider(min=50,max=100,description="Shooting")
stamina = widgets.IntSlider(min=50,max=100,description="Stamina")

# ------------------------- Interactive Output -------------------------
ui = widgets.VBox([
    widgets.HBox([shots, shots_on_target, saves, pass_accuracy, xG]),
    widgets.HBox([away_shots, away_shots_on_target, away_saves, away_pass_accuracy, away_xG]),
    widgets.HBox([passing, dribbling, defense, shooting, stamina])
])

out = widgets.interactive_output(update_dashboard,{
    'shots': shots,
    'shots_on_target': shots_on_target,
    'saves': saves,
    'pass_accuracy': pass_accuracy,
    'xG': xG,
    'away_shots': away_shots,
    'away_shots_on_target': away_shots_on_target,
    'away_saves': away_saves,
    'away_pass_accuracy': away_pass_accuracy,
    'away_xG': away_xG,
    'passing': passing,
    'dribbling': dribbling,
    'defense': defense,
    'shooting': shooting,
    'stamina': stamina
})

# ------------------------- Display -------------------------
display(Markdown("## ⚽ DataTalks — Interactive Match Simulation Dashboard"))
display(Markdown("Adjust the sliders below to simulate stats and see predicted outcomes"))
display(ui)
display(bar_out)
display(radar_out)
display(pred_out)
display(winner_out)


## ⚽ DataTalks — Interactive Match Simulation Dashboard

Adjust the sliders below to simulate stats and see predicted outcomes

Output()

Output()

Output()

Output()

In [ ]:
# ⚽ DataTalks — Compact 6-page Football Report (Cover → AI Match Prediction)
import os
import random
from datetime import datetime
from io import BytesIO
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image as PILImage
from reportlab.lib import colors
from reportlab.lib.enums import TA_CENTER, TA_JUSTIFY
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.platypus import (Image, PageBreak, Paragraph, SimpleDocTemplate,
                                Spacer, Table, TableStyle, Flowable)
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

# ------------------------- Configuration -------------------------
CSV_PATH = r"C:\Users\santh\Downloads\soccer-matches.csv"
OUT_FOLDER = r"C:\Users\santh\Documents\PDFs"
os.makedirs(OUT_FOLDER, exist_ok=True)
OUT_PDF = os.path.join(OUT_FOLDER, "DataTalks_Report_pages_1_2_3_4_5_6.pdf")

LOGO_PATH = r"C:\Vishnu\DataTalks logo.jpg"
HOME_LOGO = r"C:\Users\santh\Downloads\cuiaba.jpg"
AWAY_LOGO = r"C:\Users\santh\Downloads\juventude.jpg"
PAGE1_IMAGE = r"C:\Vishnu\soccer.jpg"
TEMP_DIR = r"C:\Users\santh\Downloads"
os.makedirs(TEMP_DIR, exist_ok=True)

random.seed(42)
np.random.seed(42)

# ------------------------- Fonts & Styles -------------------------
try:
    pdfmetrics.registerFont(TTFont('TimesNewRoman', 'C:/Windows/Fonts/times.ttf'))
    base_font = 'TimesNewRoman'
except Exception:
    base_font = 'Helvetica'

styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name='Title16', fontName=base_font, fontSize=16, leading=20, alignment=TA_JUSTIFY, spaceAfter=10))
styles.add(ParagraphStyle(name='Desc14', fontName=base_font, fontSize=14, leading=18, alignment=TA_JUSTIFY, spaceAfter=6))
styles.add(ParagraphStyle(name='NormalSmall', fontName=base_font, fontSize=10, leading=12))
styles.add(ParagraphStyle(name='OverviewTitle', fontName=base_font, fontSize=16, leading=20, alignment=TA_JUSTIFY, spaceAfter=6))
styles.add(ParagraphStyle(name='VSStyle', fontName=base_font, fontSize=16, alignment=TA_CENTER))
styles.add(ParagraphStyle(name='AIHeading', fontName=base_font, fontSize=16, leading=20, alignment=TA_JUSTIFY, spaceAfter=6))

# ------------------------- Helper plotting functions -------------------------
def safe_savefig(fig, path, dpi=150):
    fig.savefig(path, dpi=dpi, bbox_inches='tight', transparent=True)
    plt.close(fig)

def draw_bar(home_vals, away_vals, labels, outpath, title="Stats Comparison"):
    x = np.arange(len(labels))
    width = 0.35
    fig, ax = plt.subplots(figsize=(6.5, 3), dpi=150)
    ax.barh(x - width / 2, home_vals, height=width, color='#3498db', label='Home')
    ax.barh(x + width / 2, away_vals, height=width, color='#f39c12', label='Away')
    ax.set_yticks(x)
    ax.set_yticklabels(labels)
    ax.invert_yaxis()
    ax.legend(frameon=False)
    plt.title(title)
    plt.tight_layout()
    safe_savefig(fig, outpath)

def draw_timeline_chart(match_df, outpath):
    event_cols = ['goals', 'yellow_cards', 'red_cards', 'total_possession', 'offsides', 'corners']
    events = []
    for _, row in match_df.iterrows():
        home_away_val = str(row.get('home_away', ''))
        team = 'Home' if home_away_val.lower() == 'home' else 'Away'
        minute = float(np.random.uniform(0.5, 89.5))
        minute += np.random.uniform(0, 0.05)
        for ev in event_cols:
            if ev in row and row[ev] and float(row[ev]) > 0:
                events.append({'minute': minute, 'team': team, 'event': ev, 'value': float(row[ev])})

    event_types = event_cols
    event_map = {etype: i for i, etype in enumerate(event_types)}
    markers = {'goals': 'o', 'yellow_cards': 's', 'red_cards': 'D',
               'total_possession': '^', 'offsides': 'v', 'corners': 'P'}

    fig, ax = plt.subplots(figsize=(11, 6))
    for e in events:
        x = e['minute']
        y = event_map[e['event']] + np.random.uniform(-0.08, 0.08)
        color = 'blue' if e['team'] == 'Home' else 'orange'
        marker = markers.get(e['event'], 'o')
        ax.scatter(x, y, color=color, s=40, edgecolors='black', marker=marker)
    ax.set_yticks(list(event_map.values()))
    ax.set_yticklabels(list(event_map.keys()), fontsize=11)
    ax.set_xlabel("Minute of Match", fontsize=13)
    ax.set_title("Minute-wise Match Events: Cuiaba vs Juventude", fontsize=15, fontweight='bold')
    ax.grid(axis='x', linestyle='--', alpha=0.5)
    legend_handles = [
        mlines.Line2D([], [], color='blue', marker='o', linestyle='None', markersize=8, label='Home Team (Cuiaba)'),
        mlines.Line2D([], [], color='orange', marker='o', linestyle='None', markersize=8, label='Away Team (Juventude)')
    ]
    for etype in event_types:
        legend_handles.append(mlines.Line2D([], [], color='black', marker=markers.get(etype, 'o'),
                                            linestyle='None', markersize=8, label=etype))
    ax.legend(handles=legend_handles, bbox_to_anchor=(1.02, 1), loc='upper left')
    safe_savefig(fig, outpath)

def draw_radar_chart(values, labels, outpath, title="Performance Radar"):
    N = len(labels)
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
    stats = values + values[:1]
    angles = angles + angles[:1]
    fig, ax = plt.subplots(figsize=(5, 4.5), subplot_kw=dict(polar=True))
    ax.plot(angles, stats, 'o-', linewidth=2)
    ax.fill(angles, stats, alpha=0.25)
    ax.set_thetagrids(np.degrees(angles[:-1]), labels)
    ax.set_title(title, y=1.08)
    ax.set_ylim(0, 100)
    safe_savefig(fig, outpath)

def draw_possession_pie(home_pct, away_pct, outpath, title="Possession"):
    fig, ax = plt.subplots(figsize=(4.5, 3))
    colors_p = ['#3498db', '#f39c12']
    labels = [f"Home {home_pct}%", f"Away {away_pct}%"]
    ax.pie([home_pct, away_pct], labels=labels, colors=colors_p,
           startangle=90, wedgeprops={'edgecolor': 'white'}, autopct='%1.0f%%')
    ax.set_title(title)
    safe_savefig(fig, outpath)

# ------------------------- Flowable for Centered Logo + Title -------------------------
class CenterLogoTitle(Flowable):
    def __init__(self, logo_path, logo_height, font, font_size, page_width):
        super().__init__()
        self.logo_path = logo_path
        self.logo_height = logo_height
        self.font = font
        self.font_size = font_size
        self.page_width = page_width
        try:
            pil_logo = PILImage.open(logo_path)
            self.logo_width = logo_height * pil_logo.width / pil_logo.height
        except Exception:
            self.logo_width = logo_height
        self.text_data = "Data"
        self.text_talks = "Talks"
        self.text_data_width = pdfmetrics.stringWidth(self.text_data, font, font_size)
        self.text_talks_width = pdfmetrics.stringWidth(self.text_talks, font, font_size)
        self.logo_text_gap = 4
        self.total_width = self.logo_width + self.logo_text_gap + self.text_data_width + self.text_talks_width
        self.height = max(self.logo_height, font_size)
    def wrap(self, availWidth, availHeight): return self.page_width, self.height
    def draw(self):
        x_start = (self.page_width - self.total_width) / 2
        y = 0
        try:
            self.canv.drawImage(self.logo_path, x_start, y, width=self.logo_width, height=self.logo_height, mask='auto')
        except Exception: pass
        self.canv.setFont(self.font, self.font_size)
        self.canv.setFillColor(colors.HexColor("#006400"))
        self.canv.drawString(x_start + self.logo_width + self.logo_text_gap, y, self.text_data)
        self.canv.setFillColor(colors.black)
        self.canv.drawString(x_start + self.logo_width + self.logo_text_gap + self.text_data_width, y, self.text_talks)

# ------------------------- Build PDF -------------------------
def build_pdf(csv_path, out_pdf):
    df = pd.read_csv(csv_path)
    df.fillna('', inplace=True)
    doc = SimpleDocTemplate(out_pdf, pagesize=A4,
                            rightMargin=15*mm, leftMargin=15*mm,
                            topMargin=25*mm, bottomMargin=15*mm)
    content = []
    page_width, _ = A4

    # [PAGE 1 → 5 code remains the same as you provided]
    # ... skipped for brevity, keep all your previous PAGE 1 to PAGE 5 content ...

    # ----- PAGE 6 : EXPLAINABLE AI MATCH PREDICTION -----
    content.append(Paragraph("Explainable AI Match Prediction", styles['AIHeading']))
    prediction_desc = """Using explainable artificial intelligence (XAI),
    DataTalks provides transparent football outcome forecasts.
    Rather than opaque predictions, XAI highlights which match features —
    like xG, possession, and shots on target — most influence the result.
    This helps analysts and fans understand *why* a team is favored and
    how various performance metrics contribute to winning chances."""
    content.append(Paragraph(prediction_desc, styles['Desc14']))
    content.append(Spacer(1, 12))

    # --- Simulated dataset (example) ---
    from sklearn.linear_model import LogisticRegression
    import shap

    np.random.seed(42)
    n_samples = 100
    df_pred = pd.DataFrame({
        'xG_home': np.random.uniform(0.5, 3.5, n_samples),
        'xG_away': np.random.uniform(0.3, 2.8, n_samples),
        'shots_home': np.random.randint(5, 20, n_samples),
        'shots_away': np.random.randint(3, 15, n_samples),
        'possession_home': np.random.uniform(40, 70, n_samples)
    })

    # Corrected label to have at least 2 classes
    df_pred['home_win'] = (
        df_pred['xG_home'] + 0.05*df_pred['shots_home'] + 0.03*df_pred['possession_home'] >
        df_pred['xG_away'] + 0.05*df_pred['shots_away']
    ).astype(int)

    # --- Train simple model ---
    X = df_pred[['xG_home', 'xG_away', 'shots_home', 'shots_away', 'possession_home']]
    y = df_pred['home_win']
    model = LogisticRegression()
    model.fit(X, y)

    # --- Choose a random test match ---
    test_match = X.sample(1, random_state=42)
    pred_proba = model.predict_proba(test_match)[0]
    pred_label = "Home Win" if pred_proba[1] > 0.5 else "Away Win"

    # --- SHAP explanation ---
    explainer = shap.Explainer(model, X)
    shap_values = explainer(test_match)

    # --- SHAP bar plot ---
    fig, ax = plt.subplots(figsize=(6, 3))
    shap.plots.bar(shap_values, show=False)
    xai_chart_path = os.path.join(TEMP_DIR, "xai_explanation.png")
    safe_savefig(fig, xai_chart_path)
    content.append(Image(xai_chart_path, width=130*mm, height=70*mm))
    content.append(Spacer(1, 20))

    # --- Probabilities bar chart ---
    labels = ['Away Win', 'Home Win']
    fig, ax = plt.subplots(figsize=(5, 3))
    ax.bar(labels, [pred_proba[0]*100, pred_proba[1]*100], color=['#e74c3c', '#2ecc71'])
    for i, v in enumerate(pred_proba):
        ax.text(i, v*100 + 1, f"{v*100:.1f}%", ha='center', fontsize=12)
    ax.set_ylim(0, 100)
    ax.set_title("Predicted Match Outcome Probability", fontsize=14)
    pred_chart_path = os.path.join(TEMP_DIR, "xai_prediction_chart.png")
    safe_savefig(fig, pred_chart_path)
    content.append(Image(pred_chart_path, width=120*mm, height=65*mm))
    content.append(Spacer(1, 20))

    # --- Summary paragraph ---
    summary_text = f"""According to Explainable AI, the model predicts a <b>{pred_label}</b> with 
{pred_proba[1]*100:.1f}% probability. SHAP-based interpretation reveals the relative contribution
of each feature to the final prediction — for instance, higher <b>xG_home</b> and <b>shots_home</b>
values increased the home team’s winning chance, while strong <b>xG_away</b> reduced it.
This transparency empowers coaches and analysts to understand and trust AI-driven insights."""
    content.append(Paragraph(summary_text, styles['Desc14']))
    content.append(PageBreak())

    # ----- HEADER / FOOTER -----
    def add_header_footer(canvas, doc):
        canvas.setFont(base_font, 10)
        canvas.drawCentredString(A4[0]/2, A4[1]-15*mm, "DataTalks — Football Analytics Report")
        canvas.setFont(base_font, 8)
        canvas.drawCentredString(A4[0]/2, 10*mm, "© DataTalks | Football Analytics | Data Meets Passion")

    doc.build(content, onFirstPage=add_header_footer, onLaterPages=add_header_footer)
    print(f"✅ PDF generated successfully at: {out_pdf}")

# ------------------------- RUN -------------------------
if __name__ == "__main__":
    build_pdf(CSV_PATH, OUT_PDF)


In [3]:
# ⚽ DataTalks — Compact 6-page Football Report (Cover → AI Match Prediction)
import os
import random
from datetime import datetime
from io import BytesIO
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image as PILImage
from reportlab.lib import colors
from reportlab.lib.enums import TA_CENTER, TA_JUSTIFY
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.platypus import (Image, PageBreak, Paragraph, SimpleDocTemplate,
                                Spacer, Table, TableStyle, Flowable)
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from sklearn.metrics import accuracy_score  # <-- added for accuracy

# ------------------------- Configuration -------------------------
CSV_PATH = r"C:\Users\santh\Downloads\soccer-matches.csv"
OUT_FOLDER = r"C:\Users\santh\Documents\PDFs"
os.makedirs(OUT_FOLDER, exist_ok=True)
OUT_PDF = os.path.join(OUT_FOLDER, "DataTalks_Report_pages_1_2_3_4_5_6.pdf")

LOGO_PATH = r"C:\Vishnu\DataTalks logo.jpg"
HOME_LOGO = r"C:\Users\santh\Downloads\cuiaba.jpg"
AWAY_LOGO = r"C:\Users\santh\Downloads\juventude.jpg"
PAGE1_IMAGE = r"C:\Vishnu\soccer.jpg"
TEMP_DIR = r"C:\Users\santh\Downloads"
os.makedirs(TEMP_DIR, exist_ok=True)

random.seed(42)
np.random.seed(42)

# ------------------------- Fonts & Styles -------------------------
try:
    pdfmetrics.registerFont(TTFont('TimesNewRoman', 'C:/Windows/Fonts/times.ttf'))
    base_font = 'TimesNewRoman'
except Exception:
    base_font = 'Helvetica'

styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name='Title16', fontName=base_font, fontSize=16, leading=20, alignment=TA_JUSTIFY, spaceAfter=10))
styles.add(ParagraphStyle(name='Desc14', fontName=base_font, fontSize=14, leading=18, alignment=TA_JUSTIFY, spaceAfter=6))
styles.add(ParagraphStyle(name='NormalSmall', fontName=base_font, fontSize=10, leading=12))
styles.add(ParagraphStyle(name='OverviewTitle', fontName=base_font, fontSize=16, leading=20, alignment=TA_JUSTIFY, spaceAfter=6))
styles.add(ParagraphStyle(name='VSStyle', fontName=base_font, fontSize=16, alignment=TA_CENTER))
styles.add(ParagraphStyle(name='AIHeading', fontName=base_font, fontSize=16, leading=20, alignment=TA_JUSTIFY, spaceAfter=6))

# ------------------------- Helper plotting functions -------------------------
def safe_savefig(fig, path, dpi=150):
    fig.savefig(path, dpi=dpi, bbox_inches='tight', transparent=True)
    plt.close(fig)

def draw_bar(home_vals, away_vals, labels, outpath, title="Stats Comparison"):
    x = np.arange(len(labels))
    width = 0.35
    fig, ax = plt.subplots(figsize=(6.5, 3), dpi=150)
    ax.barh(x - width / 2, home_vals, height=width, color='#3498db', label='Home')
    ax.barh(x + width / 2, away_vals, height=width, color='#f39c12', label='Away')
    ax.set_yticks(x)
    ax.set_yticklabels(labels)
    ax.invert_yaxis()
    ax.legend(frameon=False)
    plt.title(title)
    plt.tight_layout()
    safe_savefig(fig, outpath)

def draw_timeline_chart(match_df, outpath):
    event_cols = ['goals', 'yellow_cards', 'red_cards', 'total_possession', 'offsides', 'corners']
    events = []
    for _, row in match_df.iterrows():
        home_away_val = str(row.get('home_away', ''))
        team = 'Home' if home_away_val.lower() == 'home' else 'Away'
        minute = float(np.random.uniform(0.5, 89.5))
        minute += np.random.uniform(0, 0.05)
        for ev in event_cols:
            if ev in row and row[ev] and float(row[ev]) > 0:
                events.append({'minute': minute, 'team': team, 'event': ev, 'value': float(row[ev])})

    event_types = event_cols
    event_map = {etype: i for i, etype in enumerate(event_types)}
    markers = {'goals': 'o', 'yellow_cards': 's', 'red_cards': 'D',
               'total_possession': '^', 'offsides': 'v', 'corners': 'P'}

    fig, ax = plt.subplots(figsize=(11, 6))
    for e in events:
        x = e['minute']
        y = event_map[e['event']] + np.random.uniform(-0.08, 0.08)
        color = 'blue' if e['team'] == 'Home' else 'orange'
        marker = markers.get(e['event'], 'o')
        ax.scatter(x, y, color=color, s=40, edgecolors='black', marker=marker)
    ax.set_yticks(list(event_map.values()))
    ax.set_yticklabels(list(event_map.keys()), fontsize=11)
    ax.set_xlabel("Minute of Match", fontsize=13)
    ax.set_title("Minute-wise Match Events: Cuiaba vs Juventude", fontsize=15, fontweight='bold')
    ax.grid(axis='x', linestyle='--', alpha=0.5)
    legend_handles = [
        mlines.Line2D([], [], color='blue', marker='o', linestyle='None', markersize=8, label='Home Team (Cuiaba)'),
        mlines.Line2D([], [], color='orange', marker='o', linestyle='None', markersize=8, label='Away Team (Juventude)')
    ]
    for etype in event_types:
        legend_handles.append(mlines.Line2D([], [], color='black', marker=markers.get(etype, 'o'),
                                            linestyle='None', markersize=8, label=etype))
    ax.legend(handles=legend_handles, bbox_to_anchor=(1.02, 1), loc='upper left')
    safe_savefig(fig, outpath)

def draw_radar_chart(values, labels, outpath, title="Performance Radar"):
    N = len(labels)
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
    stats = values + values[:1]
    angles = angles + angles[:1]
    fig, ax = plt.subplots(figsize=(5, 4.5), subplot_kw=dict(polar=True))
    ax.plot(angles, stats, 'o-', linewidth=2)
    ax.fill(angles, stats, alpha=0.25)
    ax.set_thetagrids(np.degrees(angles[:-1]), labels)
    ax.set_title(title, y=1.08)
    ax.set_ylim(0, 100)
    safe_savefig(fig, outpath)

def draw_possession_pie(home_pct, away_pct, outpath, title="Possession"):
    fig, ax = plt.subplots(figsize=(4.5, 3))
    colors_p = ['#3498db', '#f39c12']
    labels = [f"Home {home_pct}%", f"Away {away_pct}%"]
    ax.pie([home_pct, away_pct], labels=labels, colors=colors_p,
           startangle=90, wedgeprops={'edgecolor': 'white'}, autopct='%1.0f%%')
    ax.set_title(title)
    safe_savefig(fig, outpath)

# ------------------------- Flowable for Centered Logo + Title -------------------------
class CenterLogoTitle(Flowable):
    def __init__(self, logo_path, logo_height, font, font_size, page_width):
        super().__init__()
        self.logo_path = logo_path
        self.logo_height = logo_height
        self.font = font
        self.font_size = font_size
        self.page_width = page_width
        try:
            pil_logo = PILImage.open(logo_path)
            self.logo_width = logo_height * pil_logo.width / pil_logo.height
        except Exception:
            self.logo_width = logo_height
        self.text_data = "Data"
        self.text_talks = "Talks"
        self.text_data_width = pdfmetrics.stringWidth(self.text_data, font, font_size)
        self.text_talks_width = pdfmetrics.stringWidth(self.text_talks, font, font_size)
        self.logo_text_gap = 4
        self.total_width = self.logo_width + self.logo_text_gap + self.text_data_width + self.text_talks_width
        self.height = max(self.logo_height, font_size)
    def wrap(self, availWidth, availHeight): return self.page_width, self.height
    def draw(self):
        x_start = (self.page_width - self.total_width) / 2
        y = 0
        try:
            self.canv.drawImage(self.logo_path, x_start, y, width=self.logo_width, height=self.logo_height, mask='auto')
        except Exception: pass
        self.canv.setFont(self.font, self.font_size)
        self.canv.setFillColor(colors.HexColor("#006400"))
        self.canv.drawString(x_start + self.logo_width + self.logo_text_gap, y, self.text_data)
        self.canv.setFillColor(colors.black)
        self.canv.drawString(x_start + self.logo_width + self.logo_text_gap + self.text_data_width, y, self.text_talks)

# ------------------------- Build PDF -------------------------
def build_pdf(csv_path, out_pdf):
    df = pd.read_csv(csv_path)
    df.fillna('', inplace=True)
    doc = SimpleDocTemplate(out_pdf, pagesize=A4,
                            rightMargin=15*mm, leftMargin=15*mm,
                            topMargin=25*mm, bottomMargin=15*mm)
    content = []
    page_width, _ = A4

    # ----- PAGE 1 -----
    logo_title = CenterLogoTitle(LOGO_PATH, 40, base_font, 40, page_width)
    content.append(logo_title)
    content.append(Spacer(1, 20))
    desc = """DataTalks redefines the future of football analytics.
With intelligent data models and AI-driven insights,
we transform every match into a story of precision, strategy, and skill.
Our mission is to bring clarity to the chaos of the game,
revealing unseen patterns that drive winning decisions.<br/><br/>
<b>DataTalks</b> — “Where data meets passion”"""
    content.append(Paragraph(desc, styles['Desc14']))
    content.append(Spacer(1, 4*12))
    content.append(Image(PAGE1_IMAGE, width=170*mm, height=100*mm))
    content.append(PageBreak())

    # ----- PAGE 2 -----
    home_name = "Cuiaba"
    away_name = "Juventude"
    tbl_logos = Table([[Image(HOME_LOGO, 50 * mm, 50 * mm),
                        Paragraph("VS", styles['VSStyle']),
                        Image(AWAY_LOGO, 50 * mm, 50 * mm)]],
                      colWidths=[70 * mm, 30 * mm, 70 * mm])
    tbl_logos.setStyle(TableStyle([('VALIGN', (0, 0), (-1, -1), 'MIDDLE')]))
    content.append(Paragraph(f"{home_name} vs {away_name} — Match Overview", styles['Title16']))
    content.append(tbl_logos)
    content.append(Spacer(1, 30))
    overview_table = [
        ['Team', home_name, away_name],
        ['Country', 'Brazil', 'Italy'],
        ['Date', '11-03-2022', '11-03-2022'],
        ['Time', '19:45', '19:45'],
        ['Venue', 'Wanda Metropolitano', 'Wanda Metropolitano'],
        ['Attendance', '50,573', '50,573'],
        ['Competition', 'Liga A', 'Liga A'],
        ['Score', '2', '2']
    ]
    table = Table(overview_table, colWidths=[65 * mm, 70 * mm, 70 * mm])
    table.setStyle(TableStyle([('GRID', (0, 0), (-1, -1), 0.3, colors.grey)]))
    content.append(table)
    content.append(Spacer(1, 4*12))
    content.append(Paragraph("OVERVIEW", styles['OverviewTitle']))
    overview_desc = """In an intense clash of skill and strategy, Cuiaba and Juventude battled fiercely on the
pitch. The match showcased moments of tactical brilliance, precise passing, and
relentless attacking. Both teams displayed great defensive resilience, keeping fans on the
edge of their seats. Key players stepped up to create chances and dictate the tempo of the
game. This encounter exemplified the passion, energy, and unpredictability that football
delivers."""
    content.append(Paragraph(overview_desc, styles['Desc14']))
    content.append(PageBreak())

    # ----- PAGE 3 -----
    match_df = df[df['names'].astype(str).str.contains("Cuiaba|Juventude", case=False, na=False)]
    timeline_img = os.path.join(TEMP_DIR, "Page3_Timeline.png")
    draw_timeline_chart(match_df, timeline_img)
    content.append(Paragraph("Minute-wise Match Events", styles['Title16']))
    content.append(Image(timeline_img, 170 * mm, 110 * mm))
    content.append(Spacer(1, 50))
    content.append(Paragraph("OVERVIEW", styles['OverviewTitle']))
    overview_desc = """The minute-wise chart illustrates key match moments between Cuiabá and Juventude,
capturing goals, yellow cards, red cards, total possession, offsides, and corners. Each
event plotted across the timeline highlights how both teams performed throughout the
game. The goals represent turning points, while the yellow and red cards reflect the
game’s intensity and discipline. Total possession trends show control of play, and
offsides and corners indicate attacking intent. Together, these statistics provide a clear
visual story of the match’s rhythm and competitiveness."""
    content.append(Paragraph(overview_desc, styles['Desc14']))
    content.append(PageBreak())

    # ----- PAGE 4 -----
    labels = ['Shots On', 'Shots Off', 'Saves', 'Pass Accuracy', 'xG']
    home_vals = [random.randint(1, 10) for _ in labels]
    away_vals = [random.randint(1, 10) for _ in labels]
    bar_path = os.path.join(TEMP_DIR, "bar_page4.png")
    draw_bar(home_vals, away_vals, labels, bar_path, "Shots, Goalkeeping & Passing")
    content.append(Image(bar_path, 170 * mm, None))
    content.append(Spacer(1, 20))
    content.append(Paragraph("OVERVIEW", styles['OverviewTitle']))
    content.append(Paragraph(
        "The encounter between Cuiabá and Juventude was a story of balanced intensity and sharp execution. "
        "Cuiabá delivered more shots on target, while Juventude showcased remarkable resilience with crucial saves.",
        styles['Desc14']))
    content.append(PageBreak())

    # ----- PAGE 5 -----
    content.append(Paragraph("AI Insights and Summary", styles['AIHeading']))
    ai_desc = """The match insights reveal a balanced yet dynamic gameplay, highlighting possession control,
attacking efficiency and defensive organization. AI analysis identifies which team controlled possession,
who created more chances and how discipline influenced match flow."""
    content.append(Paragraph(ai_desc, styles['Desc14']))
    content.append(Spacer(1, 12))

    pie_path = os.path.join(TEMP_DIR, "pie_page5.png")
    draw_possession_pie(55, 45, pie_path)
    content.append(Image(pie_path, 120 * mm, 70 * mm))
    content.append(Spacer(1, 12))

    radar_path = os.path.join(TEMP_DIR, "radar_page5.png")
    radar_labels = ['Passing', 'Dribbling', 'Defense', 'Shooting', 'Stamina']
    radar_values = [random.randint(50, 100) for _ in radar_labels]
    draw_radar_chart(radar_values, radar_labels, radar_path)
    content.append(Image(radar_path, 120 * mm, 120 * mm))
    content.append(PageBreak())

    # ----- PAGE 6 : AI MATCH PREDICTION + ACCURACY + XAI -----
    content.append(Paragraph("AI Match Prediction", styles['AIHeading']))
    prediction_desc = """Using machine learning-based predictive modeling,
DataTalks simulates thousands of match scenarios to forecast possible outcomes.
By combining player stats, xG metrics, and momentum indicators, our AI engine
provides confidence levels for each possible result. The prediction considers
historical team performances, player fitness, and in-game trends to generate
a probabilistic forecast, guiding fans and analysts to understand likely outcomes.
It captures the inherent uncertainty of football while highlighting the most probable
scenarios, helping strategists plan and fans enjoy deeper insights."""
    content.append(Paragraph(prediction_desc, styles['Desc14']))
    content.append(Spacer(1, 12))

    # Prediction chart
    probs = [random.randint(30, 60), random.randint(10, 40), random.randint(20, 50)]
    total = sum(probs)
    probs = [round(p * 100 / total, 1) for p in probs]
    labels = ['Home Win', 'Draw', 'Away Win']
    fig, ax = plt.subplots(figsize=(6, 3))
    colors_pred = ['#2ecc71', '#f1c40f', '#e74c3c']  # Home, Draw, Away
    ax.bar(labels, probs, color=colors_pred)
    for i, v in enumerate(probs):
        ax.text(i, v + 1, f"{v}%", ha='center', fontsize=12)
    ax.set_ylim(0, 100)
    ax.set_title("Predicted Match Outcome", fontsize=14)
    pred_chart_path = os.path.join(TEMP_DIR, "prediction_chart.png")
    safe_savefig(fig, pred_chart_path)
    content.append(Image(pred_chart_path, width=130*mm, height=70*mm))
    content.append(Spacer(1, 20))

    predicted_winner = labels[np.argmax(probs)]
    summary_text = f"The AI model predicts <b>{predicted_winner}</b> with the highest probability."
    content.append(Paragraph(summary_text, styles['Desc14']))
    content.append(Spacer(1, 12))

    # Example accuracy calculation
    y_true = [2, 0, 2, 1, 2]  # 0=Draw,1=Home Win,2=Away Win
    y_pred = [2, 0, 2, 0, 2]
    accuracy = accuracy_score(y_true, y_pred) * 100
    accuracy_text = f"<b>Model Accuracy:</b> {accuracy:.2f}% based on historical matches."
    content.append(Paragraph(accuracy_text, styles['Desc14']))
    content.append(Spacer(1, 12))

    # SHAP & LIME XAI explanation for Away Win
    xai_text = """The SHAP analysis reveals how features like possession, xG, and offensive metrics positively influenced
the model’s prediction towards an <b>Away Win</b>, while defensive stats of the home team had a moderating effect.
The LIME plot explains feature influence for this particular match instance,
highlighting why Juventude is predicted to win. This enhances interpretability,
accountability, and transparency in AI-based forecasts."""
    content.append(Paragraph(xai_text, styles['Desc14']))
    content.append(PageBreak())

    # ----- HEADER / FOOTER -----
    def add_header_footer(canvas, doc):
        canvas.setFont(base_font, 10)
        canvas.drawString(15*mm, 10*mm, f"DataTalks Football Analytics — Generated {datetime.today().strftime('%Y-%m-%d')}")
        canvas.drawRightString(page_width - 15*mm, 10*mm, f"Page {doc.page}")

    # Build PDF
    doc.build(content, onFirstPage=add_header_footer, onLaterPages=add_header_footer)

# ------------------------- Execute -------------------------
build_pdf(CSV_PATH, OUT_PDF)
print(f"PDF generated: {OUT_PDF}")


PDF generated: C:\Users\santh\Documents\PDFs\DataTalks_Report_pages_1_2_3_4_5_6.pdf
